# Tutorial week 10 - Deep Learning and Convolutional Neural Networks (CNNs)

#### Run the provided code and answer the questions.

#### Note: Perform the steps described in 6COSC020W_TutorialWeek8.pdf (Week 8) if you have issues with TensorFlow.
#### 1) Download the Cifar10 dataset. This tutorial uses the test set as a validation set. Ideally we would have another independent set to test the final accuracy.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import PIL
import seaborn as sns

seed = 123 # to ensure we always get the same results
np.random.seed(seed) # to ensure we always get the same results
tf.keras.utils.set_random_seed(seed) # to ensure we always get the same results

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_labels = np.array(train_labels).transpose()[0] # convert from a column to row
test_labels = np.array(test_labels).transpose()[0] # convert from a column to row

#normalisation
train_images, test_images = train_images / 255.0, test_images / 255.0 # all values between 0 and 1 to improve performance

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(3, 3))
plt.imshow(train_images[0])
plt.title(("Label: " + str(class_names[train_labels[0]])) + " [" + str(train_labels[0]) +"]")
plt.colorbar()
plt.grid(False)
plt.show()


#### 2) Images in the training and validation sets:

In [ ]:
print('Number of images in training: ' + str(len(train_images)))
print('Number of images in testing: ' + str(len(test_images)))


#### 3) Images for each class in the training set:

In [ ]:
for i in range(len(class_names)):
    print(class_names[i] + ': ' + str(np.count_nonzero(train_labels == i)))


#### **Question 1: Know your data. Is it a balanced or imbalanced dataset?**

YOUR ANSWER: Balanced

#### 4) Display some images from the training dataset
Images provided have a small resolution to make the training faster.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(str(class_names[train_labels[i]]) + " [" + str(train_labels[i]) +"]")
    
plt.show()

#### 5) We create a simple neural network with 3 convolutional layers, with max pooling and average pooling, and 1 dense layer:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)), # Convolutional layer 3x3 with a ReLu activation
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2), # Max pooling 2x2
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'), # Convolutional layer 3x3 with a ReLu activation
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2), # Max pooling 2x2
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'), # Convolutional layer 3x3 with a ReLu activation
    tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2), # Average pooling 2x2
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10) # number of outputs = 10 (10 classess)
])

#### 6) We compile the model. We use Adam optimiser, and a loss function SparseCategoricalCrossentropy (because it is a classification task):

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

#### **Question 2: How many trainable parameters has the first convolutional layer?**

YOUR ANSWER: 26154

#### **Question 3: How many trainable parameters have the pooling layers? Why?**

YOUR ANSWER: 0, because they don't have any weights

<!-- Reference: https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d#:~:text=To%20calculate%20the%20learnable%20parameters,k%20in%20the%20current%20layer.  -->

#### 7) Train the model

In [ ]:
epochs = 15
history = model.fit(train_images, train_labels, validation_data=[test_images, test_labels], epochs=epochs)

#### 8) Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

#### 9) Get probabilities. For each test image will tell us the probability to belong to each of the 10 classes (i.e., for each test image will output an array of 10 values).
Since our model returns the logits, we add a Softmax layer in order to convert logits to probabilities.

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
probabilities = probability_model.predict(test_images)
print('Size of variable probabilities: ' + str(probabilities.shape)) # You can see the size of the arrays here (10000, 10)
print('Probabilities:')
print(probabilities) # Returns an array of 10000 arrays with 10 probabilities each (one for each class)

#### 10) Get predicted class
For each image, we have a vector of 10 probabilities (1 for each possible class) that tells us the probability that that image belongs to the class. We now want to get the class with the highest probability. We use argmax to get the class with the highest probability

In [ ]:
predictions = np.argmax(probabilities, axis = 1) # gets the maximum probability of each image (maximum value)

print('Size of variable predictions: ' + str(predictions.shape)) # We now have one value (class) for each image.
print('Predictions:')
print(predictions)
print('Labels:')
print(test_labels)

#### 11) Show the confusion matrix
The confusion matrix is a matrix that shows for each class (labels) how many images where predicted in the correct or wrong class. The diagonal of the matrix shows the classes that where predicted correctly, the rest are errors. They are useful to know the classes in which the network gets 'confused'. Label is the original (true) label, so the correct class. Prediction is the predicted class by the network (which may be correct or incorrect).

In [ ]:
def plot_confusion_matrix(actual, predicted, labels):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(7, 7)})
  ax.set_title('Confusion matrix')
  ax.set_xlabel('Prediction')
  ax.set_ylabel('Label')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0) 
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

plot_confusion_matrix(test_labels, predictions, class_names)

#### **Question 4: How many images of airplanes where predicted correctly?**

YOUR ANSWER:766

#### **Question 5: How many images of cats where predicted as frogs?**

YOUR ANSWER: 28

#### **Question 6: Which are the 2 classes with the highest number of wrong predictions? i.e., the class that gets 'confused' most. Why do you think is that?**

YOUR ANSWER:

#### **OPTIONAL - Challenge**

#### Use the code below to create a ResNet model and train it. Compare your results.
<font color='red'>**NOTE**: Each epoch takes ~20 min to run.</font>


In [ ]:
#train_labels_categorical = tf.keras.utils.to_categorical(train_labels)
#test_labels_categorical = tf.keras.utils.to_categorical(test_labels)

#print(train_labels_categorical.shape)
#print(test_labels_categorical.shape)

ResNet_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3), classes=10)
ResNet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#ResNet_model.summary()
epochs_resnet = 10
history = ResNet_model.fit(train_images, train_labels, validation_data=[test_images, test_labels], batch_size=128, epochs=epochs_resnet)